**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [5]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [6]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [7]:
classifier.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'hinge',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': 0,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [88]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    #'max_iter' : np.arange(5,10),
    'max_iter' : np.array([5, 10, 20, 50, 100]),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [89]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [90]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [91]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 3.55 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score='raise-deprecating',
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=...
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                 

In [92]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='squared_hinge',
              max_iter=50, n_iter_no_change=5, n_jobs=None, penalty='l1',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [93]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9714285714285714
{'alpha': 0.0007750000000000001, 'loss': 'squared_hinge', 'max_iter': 50, 'penalty': 'l1'}


In [94]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00150015, 0.00100005, 0.00110009, 0.00080006, 0.00100005,
        0.001     , 0.00100012, 0.00090008, 0.00100002, 0.00090003,
        0.00090005, 0.00100005, 0.00110002, 0.00100007, 0.00110002,
        0.00100009, 0.00140011, 0.00110002, 0.00140009, 0.00120013,
        0.00090005, 0.00080006, 0.00110009, 0.00100005, 0.00100009,
        0.00100009, 0.00120006, 0.00100005, 0.00110006, 0.00090008,
        0.00100002, 0.00090005, 0.00090003, 0.00100005, 0.00120006,
        0.00110009, 0.00170014, 0.0013001 , 0.00270019, 0.00200007,
        0.00090003, 0.00070007, 0.00100007, 0.00100007, 0.00100005,
        0.00100012, 0.00110004, 0.00110009, 0.00110009, 0.00109999,
        0.00090003, 0.00079999, 0.00100007, 0.001     , 0.00160005,
        0.00110002, 0.00100005, 0.00110002, 0.00100009, 0.00130012,
        0.00100002, 0.00090005, 0.00110006, 0.00089998, 0.00100012,
        0.00100009, 0.00120001, 0.00100002, 0.00140004, 0.00110011,
        0.00100002, 0.00130005,

In [95]:
grid_cv.cv_results_['mean_test_score']

array([0.65714286, 0.62380952, 0.78571429, 0.76190476, 0.88571429,
       0.86666667, 0.86190476, 0.81428571, 0.86190476, 0.81428571,
       0.6952381 , 0.74285714, 0.75238095, 0.7047619 , 0.79047619,
       0.77619048, 0.82380952, 0.8       , 0.84285714, 0.8       ,
       0.71428571, 0.7       , 0.72380952, 0.76666667, 0.82857143,
       0.71428571, 0.9       , 0.65714286, 0.89047619, 0.65714286,
       0.32857143, 0.3047619 , 0.2952381 , 0.32380952, 0.31428571,
       0.31904762, 0.33333333, 0.38571429, 0.34285714, 0.33809524,
       0.76666667, 0.68571429, 0.84285714, 0.7       , 0.87142857,
       0.8       , 0.86666667, 0.81904762, 0.86666667, 0.81904762,
       0.77142857, 0.69047619, 0.83809524, 0.6952381 , 0.93809524,
       0.71428571, 0.83809524, 0.7952381 , 0.83809524, 0.7952381 ,
       0.64285714, 0.60952381, 0.84285714, 0.67619048, 0.93809524,
       0.72380952, 0.88571429, 0.81428571, 0.88571429, 0.82380952,
       0.31904762, 0.33333333, 0.33809524, 0.37619048, 0.3    

In [96]:
print(max(grid_cv.cv_results_['mean_test_score']))
print(min(grid_cv.cv_results_['mean_test_score']))

0.9714285714285714
0.2857142857142857


Обучаем лучший классификатор

In [102]:
best_estimator = grid_cv.best_estimator_
best_estimator.fit(train_data, train_labels)
best_estimator.score(test_data, test_labels)

0.9333333333333333

#### Randomized grid search

In [79]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [80]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 340 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   error_score='raise-deprecating',
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n...
                                           tol=0.001, validation_fraction=0.1,
                                           verbose=0, warm_start=False),
                   iid='warn', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': array([0.0001  , 0.000

In [81]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666667
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [82]:
good_classifier = randomized_grid_cv.best_estimator_
good_classifier.fit(train_data, train_labels)
print(good_classifier.score(test_data, test_labels))

0.6222222222222222


In [73]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 2, 
                                                   random_state = 0)
randomized_grid_cv.fit(train_data, train_labels)
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.7904761904761904
{'penalty': 'l1', 'max_iter': 6, 'loss': 'log', 'alpha': 0.001}


In [74]:
not_so_good_classifier = randomized_grid_cv.best_estimator_
not_so_good_classifier.fit(train_data, train_labels)
print(not_so_good_classifier.score(test_data, test_labels))

0.8


In [70]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

#print check(current_time + '\n' + "Control sum: " + str(control_sum(current_time)))

2019-11-27 21:58:13
Control sum: 953
